# Introduction

**[Connect Four](https://en.wikipedia.org/wiki/Connect_Four)** is a game where two players alternate turns dropping colored discs into a vertical grid. Each player uses a different color (usually red or yellow), and the objective of the game is to be the first player to get four discs in a row. 

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/40B1MGc.png"><br/>
</center>

In this course, you will build your own intelligent agents to play the game.
- In the first lesson, you'll learn how to set up the game environment and create your first agent.
- The next two lessons focus on traditional methods for building game AI.  These agents will be smart enough to defeat many novice players!
- In the final lesson, you'll experiment with cutting-edge algorithms from the field of reinforcement learning.  The agents that you build will come up with gameplay strategies much like humans do: gradually, and with experience. 

# Join the competition

Throughout the course, you'll test your agents' performance by competing against agents that other users have created.  

To join the competition, open a new window with **[the competition page](https://www.kaggle.com/c/connectx/overview)**, and click on the **"Join Competition"** button. (_If you see a "Submit Agent" button instead of a "Join Competition" button, you have already joined the competition, and don't need to do so again._)

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/dDX1YVW.png" width=80%><br/>
</center>
    
This takes you to the rules acceptance page. You must accept the competition rules in order to participate. These rules govern how many submissions you can make per day, the maximum team size, and other competition-specific details. Then, click on **"I Understand and Accept"** to indicate that you will abide by the competition rules.

# Getting started

The game environment comes equipped with agents that have already been implemented for you.  To see a list of these default agents, run:

In [1]:
from kaggle_environments import make, evaluate

# Create the game environment
# Set debug=True to see the errors if your agent refuses to run
env = make("connectx", debug=True)

# List of available default agents
print(list(env.agents))

Loading environment lux_ai_s2 failed: No module named 'vec_noise'
['random', 'negamax']


The `"random"` agent selects (uniformly) at random from the set of **valid moves**.  In Connect Four, a move is considered valid if there's still space in the column to place a disc (i.e., if the board has seven rows, the column has fewer than seven discs).

In the code cell below, this agent plays one game round against a copy of itself.

In [2]:
# Two random agents play one game round
env.run(["random", "random"])

# Show the game
env.render(mode="ipython")

You can use the player above to view the game in detail: every move is captured and can be replayed.  _Try this now!_

As you'll soon see, this information will prove incredibly useful for brainstorming ways to improve our agents.

# Defining agents

To participate in the competition, you'll create your own agents.  

Your agent should be implemented as a Python function that accepts two arguments: `obs` and `config`.  It returns an integer with the selected column, where indexing starts at zero.  So, the returned value is one of 0-6, inclusive.

We'll start with a few examples, to provide some context.  In the code cell below:
- The first agent behaves identically to the `"random"` agent above.
- The second agent always selects the middle column, whether it's valid or not!  Note that if any agent selects an invalid move, it loses the game.
- The third agent selects the leftmost valid column.

In [3]:

import random
import numpy as np

In [4]:
# Selects random valid column
def agent_random(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return random.choice(valid_moves)

# Selects middle column
def agent_middle(obs, config):
    return config.columns//2

# Selects leftmost valid column
def agent_leftmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return valid_moves[0]

So, what are `obs` and `config`, exactly?

### `obs`

`obs` contains two pieces of information:
- `obs.board` - the game board (a Python list with one item for each grid location)
- `obs.mark` - the piece assigned to the agent (either `1` or `2`)

`obs.board` is a Python list that shows the locations of the discs, where the first row appears first, followed by the second row, and so on. We use `1` to track player 1's discs, and `2` to track player 2's discs.  For instance, for this game board:

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/kSYx4Nx.png" width=25%><br/>
</center>

`obs.board` would be `[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0]`.

### `config` 

`config` contains three pieces of information:
- `config.columns` - number of columns in the game board (`7` for Connect Four)
- `config.rows` - number of rows in the game board (`6` for Connect Four)
- `config.inarow` - number of pieces a player needs to get in a row in order to win (`4` for Connect Four)

Take the time now to investigate the three agents we've defined above.  Make sure that the code makes sense to you!

# Evaluating agents

To have the custom agents play one game round, we use the same `env.run()` method as before.

In [5]:
# Agents play one game round
env.run([agent_leftmost, agent_random])

# Show the game
env.render(mode="ipython")

The outcome of a single game is usually not enough information to figure out how well our agents are likely to perform.  To get a better idea, we'll calculate the win percentages for each agent, averaged over multiple games.  For fairness, each agent goes first half of the time.

To do this, we'll use the `get_win_percentages()` function (defined in a hidden code cell). 

In [6]:

def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

Which agent do you think performs better against the random agent: the agent that always plays in the middle (`agent_middle`), or the agent that chooses the leftmost valid column (`agent_leftmost`)?  Let's find out!

In [7]:
get_win_percentages(agent1=agent_middle, agent2=agent_random)

Agent 1 Win Percentage: 0.65
Agent 2 Win Percentage: 0.0
Number of Invalid Plays by Agent 1: 35
Number of Invalid Plays by Agent 2: 0


In [8]:
get_win_percentages(agent1=agent_leftmost, agent2=agent_random)

Agent 1 Win Percentage: 0.82
Agent 2 Win Percentage: 0.18
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


It looks like the agent that chooses the leftmost valid column performs best!

# Your turn

These agents are quite simple.  As the course progresses, you'll create increasingly complex agents!  Continue to **[make your first competition submission](https://www.kaggle.com/kernels/fork/7677818)**.

# Introduction

Even if you're new to Connect Four, you've likely developed several game-playing strategies.  In this tutorial, you'll learn to use a **heuristic** to share your knowledge with the agent.  

# Game trees

As a human player, how do you think about how to play the game?  How do you weigh alternative moves?

You likely do a bit of forecasting.  For each potential move, you predict what your opponent is likely to do in response, along with how you'd then respond, and what the opponent is likely to do then, and so on.  Then, you choose the move that you think is most likely to result in a win.

We can formalize this idea and represent all possible outcomes in a **(complete) game tree**.   

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/EZKHxyy.png"><br/>
</center>

The game tree represents each possible move (by agent and opponent), starting with an empty board.  The first row shows all possible moves the agent (red player) can make.  Next, we record each move the opponent (yellow player) can make in response, and so on, until each branch reaches the end of the game.  (_The game tree for Connect Four is quite large, so we show only a small preview in the image above_.)

Once we can see every way the game can possibly end, it can help us to pick the move where we are most likely to win.

# Heuristics

The complete game tree for Connect Four has over [4 trillion](https://oeis.org/A212693) different boards!  So in practice, our agent only works with a small subset when planning a move. 

To make sure the incomplete tree is still useful to the agent, we will use a **heuristic** (or **heuristic function**).  The heuristic assigns scores to different game boards, where we estimate that boards with higher scores are more likely to result in the agent winning the game.  You will design the heuristic based on your knowledge of the game.

For instance, one heuristic that might work reasonably well for Connect Four looks at each group of four adjacent locations in a (horizontal, vertical, or diagonal) line and assigns:
- **1000000 (`1e6`) points** if the agent has four discs in a row (the agent won), 
- **1 point** if the agent filled three spots, and the remaining spot is empty (the agent wins if it fills in the empty spot), and
- **-100 points** if the opponent filled three spots, and the remaining spot is empty (the opponent wins by filling in the empty spot).

This is also represented in the image below.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/vzQa4ML.png" width=70%><br/>
</center>

And how exactly will the agent use the heuristic?  Consider it's the agent's turn, and it's trying to plan a move for the game board shown at the top of the figure below.  There are seven possible moves (one for each column).  For each move, we record the resulting game board.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/PtnLOHt.png" width=100%><br/>
</center>

Then we use the heuristic to assign a score to each board.  To do this, we search the grid and look for all occurrences of the pattern in the heuristic, similar to a [word search](https://en.wikipedia.org/wiki/Word_search) puzzle.  Each occurrence modifies the score.  For instance,
- The first board (where the agent plays in column 0) gets a score of 2.  This is because the board contains two distinct patterns that each add one point to the score (where both are circled in the image above).  
- The second board is assigned a score of 1.
- The third board (where the agent plays in column 2) gets a score of 0.  This is because none of the patterns from the heuristic appear in the board.

The first board receives the highest score, and so the agent will select this move.  It's also the best outcome for the agent, since it has a guaranteed win in just one more move.  Check this in figure now, to make sure it makes sense to you!  

The heuristic works really well for this specific example, since it matches the best move with the highest score.  It is just one of many heuristics that works reasonably well for creating a Connect Four agent, and you may find that you can design a heuristic that works much better!

In general, if you're not sure how to design your heuristic (i.e., how to score different game states, or which scores to assign to different conditions), often the best thing to do is to simply take an initial guess and then play against your agent.  This will let you identify specific cases when your agent makes bad moves, which you can then fix by modifying the heuristic.

# Code

Our **one-step lookahead** agent will:
- use the heuristic to assign a score to each possible valid move, and
- select the move that gets the highest score.  (_If multiple moves get the high score, we select one at random._)

"One-step lookahead" refers to the fact that the agent looks only one step (or move) into the future, instead of deeper in the game tree.  

To define this agent, we will use the functions in the code cell below.  These functions will make more sense when we use them to specify the agent.

In [1]:

import random
import numpy as np

In [2]:
# Calculates score if agent drops piece in selected column
def score_move(grid, col, mark, config):
    next_grid = drop_piece(grid, col, mark, config)
    score = get_heuristic(next_grid, mark, config)
    return score

# Helper function for score_move: gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

# Helper function for score_move: calculates value of heuristic for grid
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp + 1e6*num_fours
    return score

# Helper function for get_heuristic: checks if window satisfies heuristic conditions
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

The one-step lookahead agent is defined in the next code cell.

In [3]:
# The agent is always implemented as a Python function that accepts two arguments: obs and config
def agent(obs, config):
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next turn
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

In the code for the agent, we begin by getting a list of valid moves.  _This is the same line of code we used in the previous tutorial!_

Next, we convert the game board to a 2D numpy array.  For Connect Four, `grid` is an array with 6 rows and 7 columns.

Then, the `score_move()` function calculates the value of the heuristic for each valid move. It uses a couple of helper functions:
- `drop_piece()` returns the grid that results when the player drops its disc in the selected column.
- `get_heuristic()` calculates the value of the heuristic for the supplied board (`grid`), where `mark` is the mark of the agent.  This function uses the `count_windows()` function, which counts the number of windows (of four adjacent locations in a row, column, or diagonal) that satisfy specific conditions from the heuristic.  Specifically, `count_windows(grid, num_discs, piece, config)` yields the number of windows in the game board (`grid`) that contain `num_discs` pieces from the player (agent or opponent) with mark `piece`, and where the remaining locations in the window are empty.  For instance, 
   - setting `num_discs=4` and `piece=obs.mark` counts the number of times the agent got four discs in a row.
   - setting `num_discs=3` and `piece=obs.mark%2+1` counts the number of windows where the opponent has three discs, and the remaining location is empty (the opponent wins by filling in the empty spot).

Finally, we get the list of columns that maximize the heuristic and select one (uniformly) at random.  

(**Note**: For this course, we decided to provide relatively slower code that was easier to follow.  After you've taken the time to understand the code above, can you see how to re-write it, to make it run much faster?  As a hint, note that the `count_windows()` function is used several times to loop over the locations in the game board.)

In the next code cell, we see the outcome of one game round against a random agent.

In [4]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run([agent, "random"])

# Show the game
env.render(mode="ipython")

Loading environment lux_ai_s2 failed: No module named 'vec_noise'


We use the `get_win_percentage()` function from the previous tutorial to check how we can expect it to perform on average.

In [5]:

def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [6]:
get_win_percentages(agent1=agent, agent2="random")

Agent 1 Win Percentage: 0.96
Agent 2 Win Percentage: 0.04
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


This agent performs much better than the random agent!

# Your turn

Continue to the exercise to **[improve the heuristic](https://www.kaggle.com/kernels/fork/8139646)**.

# Introduction

In the previous tutorial, you learned how to build an agent with one-step lookahead.  This agent performs reasonably well, but definitely still has room for improvement!  For instance, consider the potential moves in the figure below.  (_Note that we use zero-based numbering for the columns, so the leftmost column corresponds to `col=0`, the next column corresponds to `col=1`, and so on._)

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/aAYyy2I.png" width=90%><br/>
</center>

With one-step lookahead, the red player picks one of column 5 or 6, each with 50% probability.  But, column 5 is clearly a bad move, as it lets the opponent win the game in only one more turn.  Unfortunately, the agent doesn't know this, because it can only look one move into the future.  

In this tutorial, you'll use the **minimax algorithm** to help the agent look farther into the future and make better-informed decisions.

# Minimax

We'd like to leverage information from deeper in the game tree.  For now, assume we work with a depth of 3.  This way, when deciding its move, the agent considers all possible game boards that can result from  
1. the agent's move, 
2. the opponent's move, and 
3. the agent's next move.  

We'll work with a visual example.  For simplicity, we assume that at each turn, both the agent and opponent have only two possible moves.  Each of the blue rectangles in the figure below corresponds to a different game board.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BrRe7Bu.png" width=90%><br/>
</center>

We have labeled each of the "leaf nodes" at the bottom of the tree with the score from the heuristic.  (_We use made-up scores in the figure.  In the code, we'll use the same heuristic from the previous tutorial._)  As before, the current game board is at the top of the figure, and the agent's goal is to end up with a score that's as high as possible. 

But notice that the agent no longer has complete control over its score -- after the agent makes its move, the opponent selects its own move.  And, the opponent's selection can prove disastrous for the agent!  In particular, 
- If the agent chooses the left branch, the opponent can force a score of -1.  
- If the agent chooses the right branch, the opponent can force a score of +10.  

Take the time now to check this in the figure, to make sure it makes sense to you!

With this in mind, you might argue that the right branch is the better choice for the agent, since it is the less risky option.  Sure, it gives up the possibility of getting the large score (+40) that can only be accessed on the left branch, but it also guarantees that the agent gets at least +10 points.

This is the main idea behind the **minimax algorithm**: the agent chooses moves to get a score that is as high as possible, and it assumes the opponent will counteract this by choosing moves to force the score to be as low as possible.  That is, the agent and opponent have opposing goals, and we assume the opponent plays optimally.

So, in practice, how does the agent use this assumption to select a move?  We illustrate the agent's thought process in the figure below.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/bWezUC3.png" width=90%><br/>
</center>

In the example, minimax assigns the move on the left a score of -1, and the move on the right is assigned a score of +10.  So, the agent will select the move on the right. 

# Code

We'll use several functions from the previous tutorial.  These are defined in the hidden code cell below. 

In [1]:

import random
import numpy as np

# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

# Helper function for get_heuristic: checks if window satisfies heuristic conditions
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

We'll also need to slightly modify the heuristic from the previous tutorial, since the opponent is now able to modify the game board.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/vQ8b1aX.png" width=70%><br/>
</center>

In particular, we need to check if the opponent has won the game by playing a disc.  The new heuristic looks at each group of four adjacent locations in a (horizontal, vertical, or diagonal) line and assigns:
- **1000000 (`1e6`) points** if the agent has four discs in a row (the agent won), 
- **1 point** if the agent filled three spots, and the remaining spot is empty (the agent wins if it fills in the empty spot), 
- **-100 points** if the opponent filled three spots, and the remaining spot is empty (the opponent wins by filling in the empty spot), and
- **-10000 (`-1e4`) points** if the opponent has four discs in a row (the opponent won).

This is defined in the code cell below.

In [2]:
# Helper function for minimax: calculates value of heuristic for grid
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
    return score

In the next code cell, we define a few additional functions that we'll need for the minimax agent.  

In [3]:
# Uses minimax to calculate value of dropping piece in selected column
def score_move(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax(next_grid, nsteps-1, False, mark, config)
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
def is_terminal_window(window, config):
    return window.count(1) == config.inarow or window.count(2) == config.inarow

# Helper function for minimax: checks if game has ended
def is_terminal_node(grid, config):
    # Check for draw 
    if list(grid[0, :]).count(0) == 0:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal 
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if is_terminal_window(window, config):
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if is_terminal_window(window, config):
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False

# Minimax implementation
def minimax(node, depth, maximizingPlayer, mark, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return get_heuristic(node, mark, config)
    if maximizingPlayer:
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, minimax(child, depth-1, False, mark, config))
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark%2+1, config)
            value = min(value, minimax(child, depth-1, True, mark, config))
        return value

We won't describe the minimax implementation in detail, but if you want to read more technical pseudocode, here's the description [from Wikipedia](https://en.wikipedia.org/wiki/Minimax#Pseudocode).  (_Note that the pseudocode can be safely skipped!_)

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BwP9tMD.png" width=60%>
</center>

Finally, we implement the minimax agent in the competition format.  The `N_STEPS` variable is used to set the depth of the tree.

In [4]:
# How deep to make the game tree: higher values take longer to run!
N_STEPS = 3

def agent(obs, config):
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

In the next code cell, we see the outcome of one game round against a random agent.

In [5]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run([agent, "random"])

# Show the game
env.render(mode="ipython")

Loading environment lux_ai_s2 failed: No module named 'vec_noise'


And we check how we can expect it to perform on average.

In [6]:

def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [7]:
get_win_percentages(agent1=agent, agent2="random", n_rounds=50)

Agent 1 Win Percentage: 1.0
Agent 2 Win Percentage: 0.0
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


Not bad!

# Your turn

Continue to check your understanding and **[submit your own agent](https://www.kaggle.com/kernels/fork/8139647)** to the competition.

# Introduction

So far, our agents have relied on detailed information about how to play the game.  The heuristic really provides a lot of guidance about how to select moves!

In this tutorial, you'll learn how to use **reinforcement learning** to build an intelligent agent without the use of a heuristic.  Instead, we will gradually refine the agent's strategy over time, simply by playing the game and trying to maximize the winning rate.

In this notebook, we won't be able to explore this complex field in detail, but you'll learn about the big picture and explore code that you can use to train your own agent.

# Neural Networks

It's difficult to come up with a perfect heuristic.  Improving the heuristic generally entails playing the game many times, to determine specific cases where the agent could have made better choices.  And, it can prove challenging to interpret what exactly is going wrong, and ultimately to fix old mistakes without accidentally introducing new ones.

Wouldn't it be much easier if we had a more systematic way of improving the agent with gameplay experience?  

In this tutorial, towards this goal, we'll replace the heuristic with a neural network.

The network accepts the current board as input.  And, it outputs a probability for each possible move.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/KgAliYQ.png" width=90%><br/>
</center>

Then, the agent selects a move by sampling from these probabilities.  For instance, for the game board in the image above, the agent selects column 4 with 50% probability.

This way, to encode a good gameplay strategy, we need only amend the weights of the network so that _for every possible game board_, it assigns higher probabilities to better moves.

At least in theory, that's our goal.  In practice, we won't actually check if that's the case -- since remember that Connect Four has over 4 trillion possible game boards!

# Setup

How can we approach the task of amending the weights of the network, in practice?  Here's the approach we'll take in this lesson:
- After each move, we give the agent a **reward** that tells it how well it did:
  - **_If_** the agent wins the game in that move, we give it a reward of `+1`.
  - **_Else if_** the agent plays an invalid move (which ends the game), we give it a reward of `-10`.
  - **_Else if_** the opponent wins the game in its next move (i.e., the agent failed to prevent its opponent from winning), we give the agent a reward of `-1`.
  - **_Else_**, the agent gets a reward of `1/42`.
  
  
- At the end of each game, the agent adds up its reward.  We refer to the sum of rewards as the agent's **cumulative reward**.  
  - For instance, if the game lasted 8 moves (each player played four times), and the agent ultimately won, then its cumulative reward is `3*(1/42) + 1`.
  - If the game lasted 11 moves (and the opponent went first, so the agent played five times), and the opponent won in its final move, then the agent's cumulative reward is `4*(1/42) - 1`.
  - If the game ends in a draw, then the agent played exactly 21 moves, and it gets a cumulative reward of `21*(1/42)`.
  - If the game lasted 7 moves and ended with the agent selecting an invalid move, the agent gets a cumulative reward of `3*(1/42) - 10`.
  
Our goal is to find the weights of the neural network that (on average) maximize the agent's cumulative reward.  

This idea of using reward to track the performance of an agent is a core idea in the field of reinforcement learning.  Once we define the problem in this way, we can use any of a variety of reinforcement learning algorithms to produce an agent.

# Reinforcement Learning

There are many different reinforcement learning algorithms, such as DQN, A2C, and PPO, among others.  All of these algorithms use a similar process to produce an agent:

- Initially, the weights are set to random values.


- As the agent plays the game, the algorithm continually tries out new values for the weights, to see how the cumulative reward is affected, on average.  Over time, after playing many games, we get a good idea of how the weights affect cumulative reward, and the algorithm settles towards weights that performed better.  
    - _Of course, we have glossed over the details here, and there's a lot of complexity involved in this process.  For now, we focus on the big picture!_
    
    
- This way, we'll end up with an agent that tries to win the game (so it gets the final reward of `+1`, and avoids the `-1` and `-10`) and tries to make the game last as long as possible (so that it collects the `1/42` bonus as many times as it can).
    - _You might argue that it doesn't really make sense to want the game to last as long as possible -- this might result in a very inefficient agent that doesn't play obvious winning moves early in gameplay.  And, your intuition would be correct -- this will make the agent take longer to play a winning move!  The reason we include the `1/42` bonus is to help the algorithms we'll use to converge better.  Further discussion is outside of the scope of this course, but you can learn more by reading about the "temporal credit assignment problem" and "reward shaping"._
    
In the next section, we'll use the [**Proximal Policy Optimization (PPO)**](https://openai.com/blog/openai-baselines-ppo/) algorithm to create an agent.

# Code

There are a lot of great implementations of reinforcement learning algorithms online.  In this course, we'll use [Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/).

There's a bit of extra work that we need to do to make the environment compatible with Stable Baselines.  For this, we define the `ConnectFourGym` class below.  This class implements ConnectX as an [OpenAI Gym environment](http://gym.openai.com/docs/) and uses several methods:
- `reset()` will be called at the beginning of every game.  It returns the starting game board as a 2D numpy array with 6 rows and 7 columns.
- `change_reward()` customizes the rewards that the agent receives.  (_The competition already has its own system for rewards that are used to rank the agents, and this method changes the values to match the rewards system we designed._) 
- `step()` is used to play the agent's choice of action (supplied as `action`), along with the opponent's response.  It returns:
  - the resulting game board (as a numpy array), 
  - the agent's reward (from the most recent move only: one of `+1`, `-10`, `-1`, or `1/42`), and
  - whether or not the game has ended (if the game has ended, `done=True`; otherwise, `done=False`).
  
To learn more about how to define environments, check out the documentation [here](https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html).

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import gym
from kaggle_environments import make, evaluate
from gym import spaces

class ConnectFourGym(gym.Env):
    def __init__(self, agent2="random"):
        ks_env = make("connectx", debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        # Learn about spaces here: https://gymnasium.farama.org
        self.action_space = spaces.Discrete(self.columns)
        self.observation_space = spaces.Box(low=0, high=2, 
                                            shape=(1,self.rows,self.columns), dtype=int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
    def reset(self):
        self.obs = self.env.reset()
        return np.array(self.obs['board']).reshape(1,self.rows,self.columns)
    def change_reward(self, old_reward, done):
        if old_reward == 1: # The agent won the game
            return 1
        elif done: # The opponent won the game
            return -1
        else: # Reward 1/42
            return 1/(self.rows*self.columns)
    def step(self, action):
        # Check if agent's move is valid
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid: # Play the move
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -10, True, {}
        return np.array(self.obs['board']).reshape(1,self.rows,self.columns), reward, done, _

Loading environment lux_ai_s2 failed: No module named 'vec_noise'


In this notebook, we'll train an agent to beat the random agent.  We specify this opponent in the `agent2` argument below.

In [2]:
# Create ConnectFour environment 
env = ConnectFourGym(agent2="random")

The next step is to specify the architecture of the neural network.  In this case, we use a convolutional neural network.  To learn more about how to specify architectures with Stable-Baselines3, check out the documentation [here](https://stable-baselines3.readthedocs.io/en/master/).

Note that this is the neural network that outputs the probabilities of selecting each column.  Since we use the PPO algorithm (`PPO` in the code cell below), our network will also output some additional information (called the "value" of the input).  This is outside the scope of this course, but you can learn more by reading about "actor-critic networks".

In [3]:

import torch as th
import torch.nn as nn

!pip install "stable-baselines3"
from stable_baselines3 import PPO 
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# Neural network for predicting action values
class CustomCNN(BaseFeaturesExtractor):
    
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int=128):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # CxHxW images (channels first)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
)
        
# Initialize agent
model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616821 sha256=32590118967de8177f605b8b76507663d631efe02c0d04dda57df38ea3801ca1
  Stored in directory: /root/.cache/pip/wheels/d3/78/02/af51e23f21c31c0167d288296d764a22abb842ac6e8f52ebfa
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.4
    Uninstalling importlib-metadata-4.11.4:
      Successfully uninstalled importlib-metadata-4.11.4
  Attempting uninstall: gym
    Found existing installation: gym 0.23.1
    Uninstalling gym-0.23.1:
      Successfully uninstalled gym-0.23.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflict

/opt/conda/lib/python3.7/site-packages/botocore/httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


In the code cell above, the weights of the neural network are initially set to random values.

In the next code cell, we "train the agent", which is just another way of saying that we find weights of the neural network that are likely to result in the agent selecting good moves.

In [4]:
# Train agent
model.learn(total_timesteps=60000)

Finally, we specify the trained agent in the format required for the competition.

In [5]:
def agent1(obs, config):
    # Use the best model to select a column
    col, _ = model.predict(np.array(obs['board']).reshape(1, 6,7))
    # Check if selected column is valid
    is_valid = (obs['board'][int(col)] == 0)
    # If not valid, select random move. 
    if is_valid:
        return int(col)
    else:
        return random.choice([col for col in range(config.columns) if obs.board[int(col)] == 0])

In the next code cell, we see the outcome of one game round against a random agent.

In [6]:
# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run([agent1, "random"])

# Show the game
env.render(mode="ipython")

And, we calculate how it performs on average, against the random agent.

In [7]:

def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [8]:
get_win_percentages(agent1=agent1, agent2="random")

Agent 1 Win Percentage: 0.68
Agent 2 Win Percentage: 0.32
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


It's important to note that the agent that we've created here was only trained to beat the random agent, because all of its gameplay experience has been with the random agent as opponent.  

If we want to produce an agent that reliably performs better than many other agents, we have to expose our agent to these other agents during training.  To learn more about how to do this, you can read about **[self-play](https://openai.com/blog/competitive-self-play/)**.

# Learn more

This was a very quick and high-level introduction to reinforcement learning.  If you'd like to dig more deeply into this topic, we recommend checking out the following (free!) resources:
- David Silver's videos - [here](https://www.youtube.com/watch?v=2pWv7GOvuf0)
- Richard Sutton's and Andrew Barto's textbook - [here](http://www.incompleteideas.net/book/RLbook2018.pdf)
- Denny Britz's GitHub repository - [here](https://github.com/dennybritz/reinforcement-learning)
- The Deep RL Bootcamp - [here](https://sites.google.com/corp/view/deep-rl-bootcamp/lectures)

# Your turn

Continue to check your understanding and **[run the code yourself](https://www.kaggle.com/kernels/fork/8222487)**!